In [2]:
%iam_role arn:aws:iam::481237842548:role/CF-dev-TruveDevops-Glue-Role

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
It looks like there is a newer version of the kernel available. The latest version is 0.32 and you have 0.31 installed.
Please run `pip install --upgrade aws-glue-sessions` to upgrade your kernel
Current iam_role is None
iam_role has been set to arn:aws:iam::481237842548:role/CF-dev-TruveDevops-Glue-Role.


In [4]:
%region us-west-2

Previous region: None
Setting new region to: us-west-2
Region is set to: us-west-2


In [0]:
%additional_python_modules "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/sstm_transformation-0.1.0-py3-none-any.whl"
%number_of_workers 2
%connections dev-glue-redshift-connection
#%idle_timeout 30

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Additional python modules to be included:
s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/sstm_transformation-0.1.0-py3-none-any.whl
Previous number of workers: 5
Setting new number of workers to: 2
Connections to be included:
dev-glue-redshift-connection
Authenticating with profile=default
glue_role_arn defined by user: arn:aws:iam::481237842548:role/CF-dev-TruveDevops-Glue-Role
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 2
Session ID: bd5c0e92-0664-4c2e-b047-4dde3f62b06e
Applying the following default arguments:
--glue_kernel_version 0.31
--enable-glue-datacatalog true
--additional-python-modules s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/sstm_transformation-0.1.0-py3-none-any.whl
Waiting for session bd5c0e92-0664-4c2e-b047-4dde3f62b06e to get into ready status...
Session bd5c0e92-0664-4c2e-b047-4dde3f62b06e has been created




In [1]:
#download configuration file
import boto3
session = boto3.Session()

s3 = session.resource('s3')
s3.Bucket('dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o').download_file('confs/filevine/sstm.yaml', 'sstm.yaml')

In [2]:
with open("sstm.yaml", "r") as f:
    sstm_config = f.read()
    

In [3]:
def readFromFile(ext, path) :
    fileContent = spark.read.format(ext).option("header", "true").option("mergeSchema", "true").load(path)
    return fileContent


In [4]:
def writeToFile(df_join , path, mode) :
    df_join.write.parquet(path,mode=mode)

In [5]:
spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")
spark.conf.set("spark.sql.sources.partitionColumnTypeInference.enabled", "false")


In [7]:
from sstm_transformation.instagram_builder import InstagramBuilder

In [9]:
#Read All dfs 

df_age =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/instagram/*/age.parquet")
df_city =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/instagram/*/city.parquet")
df_country =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/instagram/*/country.parquet")
df_date =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/instagram/*/date.parquet")
df_gender =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/instagram/*/gender.parquet")
df_hashtag =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/instagram/*/hashtag.parquet")
df_hour =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/instagram/*/hour.parquet")
df_locale =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/instagram/*/locale.parquet")
df_mediaType =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/instagram/*/mediaType.parquet")
df_post =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/instagram/*/post.parquet")


In [10]:

builder = InstagramBuilder("sstm.yaml", spark=spark)


In [11]:
#Upload data on S3 
from awsglue.dynamicframe import DynamicFrame
def upload_result(result):
    temp_bucket = "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/temp"
    for key, df in result.items():
        print(f"Writing {key}")
        writeToFile(df,"s3://dev-truve-devops-05-databr-bucketetlprocesseddata-h2m2xopoctot/{}/".format(key.lower()), "overwrite")
        dyf = DynamicFrame.fromDF(dataframe=df, glue_ctx=glueContext, name=key)
        table_name = key.lower()
        dyf.show(10)
        #break
        redshift_connection_options = {"url": "jdbc:redshift://dev-redshift-cluster.cxorr0thbkvd.us-west-2.redshift.amazonaws.com:5439/truve", 
                        "user": "truveadmin", 
                        "password": "YuXpRNej45S1E7mT",
                        "preactions" : "delete from tsm.{}".format(table_name),
                        "dbtable": "tsm.{}".format(table_name), 
                        "redshiftTmpDir": temp_bucket} 
        datasink4 = glueContext.write_dynamic_frame.from_options(frame=dyf,
                                                        connection_type="redshift",
                                                        connection_options=redshift_connection_options)
        #break
    

In [13]:
#Age Transformation
def age_transformation(input_df):
    #Read Raw Data
    result = builder.build_age(df=input_df)
    #print(result)
    upload_result(result)
    
age_transformation(df_age)

Writing IG_Ages


In [14]:
#Age Transformation
def cities_transformation(input_df):
    #Read Raw Data
    result = builder.build_cities(df=input_df)
    #print(result)
    upload_result(result)
    
cities_transformation(df_city)

Writing IG_Cities


In [15]:
#Age Transformation
def country_transformation(input_df):
    #Read Raw Data
    result = builder.build_countries(df=input_df)
    #print(result)
    upload_result(result)
    
country_transformation(df_country)

Writing IG_Countries


In [16]:
#Age Transformation
def date_transformation(input_df):
    #Read Raw Data
    result = builder.build_dates(df=input_df)
    #print(result)
    upload_result(result)
    
date_transformation(df_date)

Writing IG_Dates


In [21]:
#Age Transformation
def gender_transformation(input_df):
    #Read Raw Data
    result = builder.build_gender(df=input_df)
    #print(result)
    upload_result(result)
    
gender_transformation(df_gender)

Writing IG_Gender


In [20]:
#Age Transformation
def post_transformation(input_df):
    #Read Raw Data
    result = builder.build_posts(df=input_df)
    #print(result)
    upload_result(result)
    
post_transformation(df_post)

Writing IG_Posts
{"post_id": "1318234281440137606", "date": 2022-07-08 13:58:58.0, "caption": "wasim akram

#wasimakram", "permalink": "https://www.instagram.com/p/CfwPpd9NxBl/", "username": "ezain801", "media_url": "https://scontent.cdninstagram.com/v/t51.29350-15/292661686_739765284013624_5543534873404267636_n.jpg?_nc_cat=109&ccb=1-7&_nc_sid=8ae9d6&_nc_ohc=5ISD2rl8dI4AX9ROHnO&_nc_ht=scontent.cdninstagram.com&edm=AM6HXa8EAAAA&oh=00_AT_adUb9nrnSWz004ETa-SbRw9deyu7qMmyWr0uOJmbX8A&oe=62CCDC33", "media_type": "IMAGE", "comments": 0, "likes": 2, "org_id": 13}
{"post_id": "1317958106675745274", "date": 2022-07-08 13:58:24.0, "caption": "federer

#federer #nft", "permalink": "https://www.instagram.com/p/CfwPlVYtT01/", "username": "ezain801", "media_url": "https://scontent.cdninstagram.com/v/t51.29350-15/292612179_755757882440080_8844275205674289291_n.jpg?_nc_cat=103&ccb=1-7&_nc_sid=8ae9d6&_nc_ohc=IG1m1y9fUwsAX_VDdWz&_nc_ht=scontent.cdninstagram.com&edm=AM6HXa8EAAAA&oh=00_AT_GznssoZqkGcESBzzD

In [22]:
#Age Transformation
def hashtag_transformation(input_df):
    #Read Raw Data
    result = builder.build_hashtags(df=input_df)
    #print(result)
    upload_result(result)
    
hashtag_transformation(df_hashtag)

Writing IG_Hashtags
{"date": "2022-07-19", "hashtag": "#wasimakram", "posts": 1, "media_count": null, "org_id": 13}
{"date": "2022-07-19", "hashtag": "#federer", "posts": 1, "media_count": null, "org_id": 13}
{"date": "2022-07-19", "hashtag": "#nft", "posts": 1, "media_count": null, "org_id": 13}
{"date": "2022-07-19", "hashtag": "#rafa", "posts": 1, "media_count": null, "org_id": 13}
{"date": "2022-07-08", "hashtag": "rafa", "posts": 34, "media_count": null, "org_id": 13}
{"date": "2022-07-08", "hashtag": "federer", "posts": 33, "media_count": null, "org_id": 13}


In [23]:
#Age Transformation
def hour_transformation(input_df):
    #Read Raw Data
    result = builder.build_hours(df=input_df)
    #print(result)
    upload_result(result)
    
hour_transformation(df_hour)

Writing IG_Hours
{"date": "2022-07-08", "hour": "18", "posts": 3, "org_id": 13}
{"date": "2022-07-08", "hour": "1", "posts": 22, "org_id": 13}
{"date": "2022-07-08", "hour": "2", "posts": 22, "org_id": 13}
{"date": "2022-07-08", "hour": "3", "posts": 22, "org_id": 13}
{"date": "2022-07-08", "hour": "4", "posts": 22, "org_id": 13}
{"date": "2022-07-08", "hour": "5", "posts": 22, "org_id": 13}
{"date": "2022-07-08", "hour": "6", "posts": 42, "org_id": 13}
{"date": "2022-07-08", "hour": "7", "posts": 23, "org_id": 13}
{"date": "2022-07-08", "hour": "8", "posts": 51, "org_id": 13}


In [24]:
#Age Transformation
def locale_transformation(input_df):
    #Read Raw Data
    result = builder.build_locals(df=input_df)
    #print(result)
    upload_result(result)
    
locale_transformation(df_locale)

Writing IG_Locales


In [25]:
#Age Transformation
def mediatype_transformation(input_df):
    #Read Raw Data
    result = builder.build_mediatypes(df=input_df)
    #print(result)
    upload_result(result)
    
mediatype_transformation(df_mediaType)

Writing IG_MediaTypes
{"date": "2022-07-08", "media_type": "IMAGE", "posts": 3, "org_id": 13}


In [9]:
%stop_session

Stopping session: bd5c0e92-0664-4c2e-b047-4dde3f62b06e
Stopped session.
